In [19]:
import numpy as np
from StringIO import StringIO
import pandas as pd
from __future__ import division
import urllib2,base64
import cPickle as pickle
from collections import Counter
from collections import defaultdict
from collections import OrderedDict
from __future__ import division
import scipy as sp
import datetime
import math
import operator
import re
import os
import os.path
import urllib, json
import csv
from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor,ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import LeavePLabelOut
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import r2_score
from scipy import stats
from sklearn.neighbors import DistanceMetric
from urllib2 import urlopen, Request
from operator import itemgetter
from scipy.spatial import KDTree
from numpy import linalg as LA
from calendar import monthrange
from sklearn import metrics
import tarfile
import dask

#from deepmoji.sentence_tokenizer import SentenceTokenizer
#from deepmoji.model_def import deepmoji_emojis
#from deepmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
#from deepmoji.model_def import deepmoji_transfer
#from deepmoji.global_variables import PRETRAINED_PATH
#from deepmoji.finetuning import (
#    load_benchmark,
#    finetune)

import dask.dataframe as dd
import dask.array as da
import dask.bag as db
import dask.multiprocessing
from dask import compute, delayed

import time
%matplotlib inline

In [20]:
YEAR="2015"

In [25]:
emoji_df = pd.read_csv(YEAR+"_emo.out",
                 sep=",", #or delim_whitespace=True, #separator is whitespace
                 header=None, #no header
                 names=['JOY','FEAR','ANGER','SADNESS','DISGUST','SHAME','GUILT'] #set columns names joy, fear, anger, sadness, disgust, shame, guilt
                 ) #parse datetime

In [26]:
emoji_df.shape

(21802675, 7)

In [23]:
ill_df=pd.read_csv('data/US_2015_integrated.csv',header=0,encoding='utf-8', index_col=False) 
#ill_df['timestamp']=pd.to_datetime(ill_df['timestamp'], errors='coerce')
ill_df.shape

(21802675, 18)

In [27]:
twe_emo_df = pd.concat([ill_df, emoji_df], axis=1)
twe_emo_df.shape

(21802675, 25)

In [29]:

pixelwidth=0.0416666666667 #PRISM cell size (800m)
width_50=pixelwidth*12.5
width_25=pixelwidth*6.25
### BOunding box for Illinois
westlon=-124.7844079
southlat=24.7433195
eastlon=-66.9513812
northlat=49.3457868

resol_lon= int(round(abs(westlon-eastlon)/width_50))
resol_lat= int(round(abs(southlat-northlat)/width_50))

n_lat, n_lon=(resol_lat, resol_lon)
lat_bins = np.linspace(southlat, northlat, n_lat)
lon_bins = np.linspace(westlon, eastlon, n_lon)

lat_labels=np.arange(resol_lat-1)
lon_labels=np.arange(resol_lon-1)

### Bin all observations
twe_emo_df = twe_emo_df.assign(
    LAT_cut=pd.cut(twe_emo_df.LAT,lat_bins,labels=False),
    LON_cut=pd.cut(twe_emo_df.LON,lon_bins,labels=False)
)

twe_emo_df=twe_emo_df.rename(index=str, columns={"count": "count_daily"})
twe_emo_df['timestamp']=pd.to_datetime(twe_emo_df['timestamp'], errors='coerce')
### Categorize on Urban or NOt
twe_emo_df.dropna(axis=0, how='any',inplace=True)
ill_urban_df=twe_emo_df[twe_emo_df['URBAN']==True]
ill_rural_df=twe_emo_df[twe_emo_df['URBAN']==False]


print "Spatial Bin on LAT/LON with resolution (n_lat, n_lon):",(resol_lat, resol_lon)
#print lat_bins
#print lon_bins
twe_emo_df.head()

Spatial Bin on LAT/LON with resolution (n_lat, n_lon): (47, 111)


,Unnamed: 0,count_daily,LAT,length_daily,LON,timestamp,txt,URBAN,TMAX,TMIN,...,length_norm,JOY,FEAR,ANGER,SADNESS,DISGUST,SHAME,GUILT,LAT_cut,LON_cut
0,0,1,25.486813,19,-80.481691,2015-01-01,"[@peeznuts, cop-, license, and, registration, ...",True,28.379999,19.469999,...,1.907277,0.089687,0.163687,0.262556,0.054682,0.112624,0.156748,0.160015,1.0,84.0
1,1,1,25.672703,7,-80.419119,2015-01-01,"[@gabssgonzalezz, @nspizale41, gaby, chill, wi...",True,27.700001,19.450001,...,0.702681,0.100029,0.097410,0.222943,0.066086,0.196843,0.182865,0.133824,1.0,84.0
2,2,1,25.728139,14,-80.242204,2015-01-01,"[#happynewyear, #nana, #2015, i, love, my, nan...",True,26.959999,20.780001,...,1.405362,0.531068,0.121486,0.070829,0.081401,0.066226,0.060988,0.068002,1.0,84.0
3,3,1,25.749842,13,-80.396288,2015-01-01,"[if, i, send, you, a, snapchat, i, did, to, ev...",True,27.290001,19.930000,...,1.304979,0.049507,0.106459,0.049856,0.016562,0.038345,0.553312,0.185959,1.0,84.0
4,4,1,25.753495,10,-80.324779,2015-01-01,"[@stefanyamaro, @kaytlinmariex3, just, sayin, ...",True,27.120001,20.440001,...,1.003830,0.309587,0.068134,0.119389,0.039704,0.158637,0.200502,0.104046,1.0,84.0


In [30]:
YEAR=2015
ill_urban_monthlygrouped_df=ill_urban_df.groupby([ill_urban_df.month,ill_urban_df.LAT_cut,ill_urban_df.LON_cut])['count_daily'].sum().to_frame().reset_index()
ill_rural_monthlygrouped_df=ill_rural_df.groupby([ill_rural_df.month,ill_rural_df.LAT_cut,ill_rural_df.LON_cut])['count_daily'].sum().to_frame().reset_index()

ill_urban_monthlygrouped_df=ill_urban_monthlygrouped_df.rename(index=str, columns={"count_daily": "monthly_cell_count_urban"})
ill_rural_monthlygrouped_df=ill_rural_monthlygrouped_df.rename(index=str, columns={"count_daily": "monthly_cell_count_rural"})

ill_urban_monthlygrouped_df['monthly_cell_daily_count'] = ill_urban_monthlygrouped_df.apply(lambda row: row['monthly_cell_count_urban'] / monthrange(YEAR,int(row.month))[1], axis=1)
ill_rural_monthlygrouped_df['monthly_cell_daily_count'] = ill_rural_monthlygrouped_df.apply(lambda row: row['monthly_cell_count_rural'] / monthrange(YEAR,int(row.month))[1], axis=1)

ill_merge_monthlygrouped_df=pd.merge(ill_urban_monthlygrouped_df, ill_rural_monthlygrouped_df, how='outer', on=['month','LAT_cut','LON_cut'], sort=True,
         suffixes=('_urban', '_rural'))

ill_merge_monthlygrouped_df.fillna(0).head() ##MONTHLY DAILY CELL COUNT

,month,LAT_cut,LON_cut,monthly_cell_count_urban,monthly_cell_daily_count_urban,monthly_cell_count_rural,monthly_cell_daily_count_rural
0,1,0.0,83.0,42.0,1.354839,11.0,0.354839
1,1,0.0,84.0,192.0,6.193548,38.0,1.225806
2,1,1.0,83.0,0.0,0.000000,30.0,0.967742
3,1,1.0,84.0,25090.0,809.354839,673.0,21.709677
4,1,2.0,49.0,234.0,7.548387,15.0,0.483871


In [31]:
ill_urban_dailygrouped_df=ill_urban_df.groupby([ill_urban_df.timestamp,ill_urban_df.LAT_cut,ill_urban_df.LON_cut]).agg({'count_daily':'sum','TMAX':'mean','TMIN':'mean','PRCP':'mean','HUM':'mean','CC':'mean','WIND':'mean','month':'first','JOY':'mean','FEAR':'mean','ANGER':'mean','SADNESS':'mean','DISGUST':'mean','SHAME':'mean','GUILT':'mean'}).reset_index()
ill_rural_dailygrouped_df=ill_rural_df.groupby([ill_rural_df.timestamp,ill_rural_df.LAT_cut,ill_rural_df.LON_cut]).agg({'count_daily':'sum','TMAX':'mean','TMIN':'mean','PRCP':'mean','HUM':'mean','CC':'mean','WIND':'mean','month':'first','JOY':'mean','FEAR':'mean','ANGER':'mean','SADNESS':'mean','DISGUST':'mean','SHAME':'mean','GUILT':'mean'}).reset_index()
#ill_rural_dailygrouped_df=ill_rural_df.groupby([ill_rural_df.timestamp,ill_rural_df.LAT_cut,ill_rural_df.LON_cut])['count_daily'].sum().to_frame().reset_index()

ill_merge_dailygrouped_df=pd.merge(ill_urban_dailygrouped_df, ill_rural_dailygrouped_df, how='outer', on=['timestamp','LAT_cut','LON_cut','month'], sort=True, suffixes=('_urban', '_rural'))

ill_merge_daily_df=pd.merge(ill_merge_dailygrouped_df,ill_merge_monthlygrouped_df, how="left", on=['LAT_cut','LON_cut','month'])

ill_merge_daily_df=ill_merge_daily_df
#f, ax = plt.subplots(1, 1, sharey=True)
#f.set_size_inches(18.5, 10.5, forward=True)

#test=ill_merge_daily_df[ill_merge_daily_df['timestamp']=='2016-02-21']
#plt.scatter(test['LON_cut'], test['LAT_cut'], s=700, c=test['PRCP_rural'], cmap="Reds", alpha=0.5,edgecolors=None,linewidths=0)
ill_merge_daily_df['timestamp']=ill_merge_daily_df['timestamp'].dt.strftime("%Y-%m-%d")
ill_merge_daily_df.head()

,timestamp,LAT_cut,LON_cut,TMIN_urban,CC_urban,month,count_daily_urban,SADNESS_urban,GUILT_urban,DISGUST_urban,...,TMAX_rural,SHAME_rural,HUM_rural,ANGER_rural,PRCP_rural,WIND_rural,monthly_cell_count_urban,monthly_cell_daily_count_urban,monthly_cell_count_rural,monthly_cell_daily_count_rural
0,2015-01-01,0.0,83.0,20.321250,49.5,1,8.0,0.051385,0.071582,0.275869,...,26.350000,0.069612,86.0,0.423173,0.000000,2.552901,42.0,1.354839,11.0,0.354839
1,2015-01-01,0.0,84.0,20.635294,49.5,1,17.0,0.104014,0.119491,0.125501,...,NaN,NaN,NaN,NaN,NaN,NaN,192.0,6.193548,38.0,1.225806
2,2015-01-01,1.0,83.0,NaN,NaN,1,NaN,NaN,NaN,NaN,...,27.724999,0.059131,86.0,0.063310,0.000000,2.552901,NaN,NaN,30.0,0.967742
3,2015-01-01,1.0,84.0,20.350864,49.5,1,1078.0,0.088062,0.120592,0.140967,...,27.254848,0.135698,86.0,0.132796,0.063636,2.552901,25090.0,809.354839,673.0,21.709677
4,2015-01-01,2.0,49.0,4.017143,96.5,1,14.0,0.079176,0.139550,0.156228,...,NaN,NaN,NaN,NaN,NaN,NaN,234.0,7.548387,15.0,0.483871


In [32]:
ill_count_daily_df=pd.read_csv('data/US_2015_50KMbinned.csv',encoding='utf-8',index_col=0)

In [33]:
print "Emo shape ", ill_merge_daily_df.shape
print "Count shape ", ill_count_daily_df.shape

Emo shape  (587214, 36)
Count shape  (587214, 31)


In [34]:
ill_count_daily_df.head()

,timestamp,LAT_cut,LON_cut,count_daily_urban,TMIN_urban,CC_urban,HUM_urban,TMAX_urban,PRCP_urban,WIND_urban,...,monthly_cell_daily_count_rural,TMAX_HIS_mean,TMIN_HIS_mean,PRCP_HIS_mean,HUM_HIS_mean,WIND_HIS_mean,CC_HIS_mean,ncount_daily_urban,ncount_daily_rural,cell_id
0,2015-01-01,0.0,83.0,8.0,20.321250,49.5,86.0,26.110000,0.000000,2.552901,...,0.354839,NaN,NaN,NaN,80.268669,7.187416,57.916634,6.645161,0.645161,"(0.0, 83.0)"
1,2015-01-01,0.0,84.0,17.0,20.635294,49.5,86.0,26.389412,0.000000,2.552901,...,1.225806,NaN,NaN,NaN,80.268669,7.187416,57.916634,10.806452,NaN,"(0.0, 84.0)"
2,2015-01-01,1.0,83.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.967742,24.817665,16.374998,1.972334,80.268669,7.187416,57.916634,NaN,1.032258,"(1.0, 83.0)"
3,2015-01-01,1.0,84.0,1078.0,20.350864,49.5,86.0,27.121699,0.296437,2.552901,...,21.709677,25.745331,15.748001,0.731333,80.268669,7.187416,57.916634,268.645161,12.290323,"(1.0, 84.0)"
4,2015-01-01,2.0,49.0,14.0,4.017143,96.5,93.0,7.911429,12.377143,4.102217,...,0.483871,NaN,NaN,NaN,70.404663,4.145273,55.726627,6.451613,NaN,"(2.0, 49.0)"


In [35]:
cols_to_use = ill_merge_daily_df.columns.difference(ill_count_daily_df.columns)
cols_to_use = cols_to_use.tolist()+ ['timestamp', 'LAT_cut','LON_cut']
print cols_to_use

['ANGER_rural', 'ANGER_urban', 'DISGUST_rural', 'DISGUST_urban', 'FEAR_rural', 'FEAR_urban', 'GUILT_rural', 'GUILT_urban', 'JOY_rural', 'JOY_urban', 'SADNESS_rural', 'SADNESS_urban', 'SHAME_rural', 'SHAME_urban', 'timestamp', 'LAT_cut', 'LON_cut']


In [36]:
ill_emo_count_daily_df = pd.merge(ill_count_daily_df, ill_merge_daily_df[cols_to_use], how='inner', on=['timestamp', 'LAT_cut','LON_cut'],validate="one_to_one")

In [37]:
print "Merge Shape ",ill_emo_count_daily_df.shape

Merge Shape  (587214, 45)


In [38]:
ill_emo_count_daily_df.to_csv('data/US_2015_EMO_50KMbinned.csv',encoding='utf-8')